## 평가지표 (Evaluation Metrics)
#### 추천 시스템 평가 관점
- **개발자 관점** : 모델의 정확도(평점, ranking)
- **비즈니스 / 서비스 관점** : 매출이나 클릭률이 얼마나 증가하는가?
- **품질 관점** : 연관성, 다양성, 새로움, 참신함

| ... | offline 평가 | online 평가 | 사용자 study |
| :---: | :---: | :---: | :---: |
| 평가 방법 | simulation ( 사용자 행동 이력에서 얻은 과거의 log를 사용해 모델의 예측 정밀도등을 평가) | 새로운 테스트 대상의 추천 모델이나 새로운 UI를 일부 사용자에게 실제로 제공 / A/B test , interleaving | 설문조사, 인터뷰, 모델, UI의 정성적인 평가 |
| 평가 항목 | MAE, RMSE / 정밀도, 재현율, F1 score / MAP, NDCG, MRR / Coverage, 다양성, 신규성, 흥미로움 | 클릭률, 매출, 지속률 등 | 만족도, 유용성, 신선함 등 |
| 장점 | 구현 비용이 낮다 | 비즈니스 지표를 직접 평가 | 사용자 만족도를 직접 조사 |
| 단점 | 비즈니스 지표와의 정합성이 없는 경우가 있다 | 구현 비용이 높다, 사용자 만족도를 떨어뜨릴 위험 | 개인 취향에 따른 대답의 분산이 큼, 조사 결과의 재현성을 보장하기 어렵다 | 


### offline 평가 지표
- 정밀도 평가 : 과거 사용자 행동을 학습하고 미지의 사용자 행동 예측
- 데이터 분리 방법
    - 평점을 분리
    - 사용자를 분리
        - 랜덤 기준, 
        - 시계열 데이터 : 시간점 기점을 기준
    - 아이템을 분리
- 모델 튜닝 : parameter tuning
    - validation data로 일반화 성능 평가
    - 이웃의 사이즈, k개의 추천리스트 개수
    - Grid search : 파라미터 후보를 미리 정하고 각 파라미터의 모든 조합에 대한 펴악
    - Bayes 최적화 : 파라미터 튜닝 정도에 따라 이전 검증 결과를 사용하여 이후 파라미터를 베이즈 확률 프레임에서 선택 (Optuna, Hyperopt)
- 평가 지표 : 학습한 모델의 성능이나 예측값의 좋고 나쁨을 측정하는 지표

    | 지표 분류 | 사용 목적 | 지표 | 
    | :---: | :---: | :---: |
    | 예측 오차 지표 | 학습 모델이 얼마나 테스트 데이터 평가값에 가깝게 예측하는지 측정 | MAE, MSE, RMSE |
    | 집합 평균 지표 | 모델이 출력한 높은 점수의 item k개에 관한 추출 능력을 측정, 클릭이나 구매 유무 등 이진 분류의 정밀도 평가나 추천 범위를 측정 | Precision, Recall, F1 score |
    | 순위 평가 지표 | item순서를 고려한 순위 평가에 사용 | 모델이 출력한 높은 점수의 item k개가 얼마나 올바르게 나열 되었는지 측정 | MAP, nDCG, MRR |
    | 기타 평가 지표 | 사용자 만족도를 직접 측정 | Coverage, 다양성, 신규성, 흥미로움 |



In [1]:
import numpy as np

from typing import List

### 예측 오차 지표
- $n$ : item의 개수, $y_i$ : 실제 평점, $\hat{y}_i$ : 예측 평점
- MAE (Mean Absolute Error, 평균 절대값 오차) : 실제 값과 예측 값의 차이의 절대값의 평균
    - $MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$
- MSE (Mean Squared Error, 평균 제곱 오차) : 실제 값과 예측 값의 차이의 제곱의 평균
    - $MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$
- RMSE (Root Mean Squared Error, 평균 제곱근 오차) : MSE의 제곱근
    - $RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$

In [3]:
r = [0, 1, 2, 3, 4]
r_hat = [0.1, 1.1, 2.1, 3.1, 4.1]
print(r)
print(r_hat)

[0, 1, 2, 3, 4]
[0.1, 1.1, 2.1, 3.1, 4.1]


In [4]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(r, r_hat))

0.09999999999999998


In [5]:
from sklearn.metrics import mean_squared_error

print(mean_squared_error(r, r_hat))

0.009999999999999995


In [6]:
print(np.sqrt(mean_squared_error(r, r_hat)))

0.09999999999999998


#### 머신러닝 평가 지표 쉽고 간단하게 설명
- True : 예측이 맞다 / False : 예측이 틀리다.
- Positive : 모델이 긍정이라고 예측한 것 / Negative : 모델이 부정이라고 예측한 것

    | | True | False |
    | :---: | :---: | :---: |
    | Positive | TP | FP |
    | Negative | FN | TN |
    
    - True Positive : 실제 True인 것을 True라고 예측
    - False Positive : 실제 False인 것을 True라고 예측
    - False Negative : 실제 True인 것을 False라고 예측
    - True Negative : 실제 False인 것을 False라고 예측

**모델의 예측 능력을 다양한 관점에서 평가**
- 정확도 : 전체 사례 중에서 모델이 **올바르게 예측**한 비율
- 특이도 : 실제 부정인 사례 중에서 모델이 **올바르게 부정**으로 예측한 비율
- 정밀도 : 모델이 긍정이라고 예측한 것 중에서 실제로 긍정인 비율
- 재현율(민감도) : 실제 긍정인 것 중에서 모델이 올바르게 긍정으로 예측한 비율
- F1-score : 정밀도와 재현율의 조화평균 , 모델이 얼마나 정확하게 긍정 사례를 예측하고, 실제 긍정 사례를 얼마나 잘 찾아냈는지를 균형 있게 평가
    - 조화평균 공식 : $ \frac{2ab}{a+b}$

### 집합 평가 지표
- 머신러닝 평가 지표로 많이 사용되는 Precision, Recall을 의미하는 것
- 하지만, 뒤에 `@K`가 붙었다. 기존의 Precision, Recall는 binary classification에 사용되지만, 추천시스템은 하나의 아이템만 추천하지 않으며 정답도 여러 개가 될 수 있다.
- **따라서, K는 추천 아이템 수를 의미한다.**

1. $Precision@K$ : 내가 추천한 아이템 K개 중에 실제 사용자가 관심있는 아이템의 비율을 의미
    - $Precision@K = \frac{|\{C \cap R_K\}|}{K}$
2. $Recall@K$ : 사용자가 관심있는 모든 아이템(집합 요소) 중에서 내가 추천한 아이템 K개가 얼마나 포함되는지 비율을 의미
    - $Recall@K = \frac{|\{C \cap R_K\}|}{|R|}$
    - $C$ : 적합 아이템 집합, $R_K$ : 예측 item 집합의 K번째 이내 item, $K$ : 순위의 길이
3. $F1-measure$ : Precision과 Recall의 조화평균
    - $F1 = \frac{2 \times Precision \times Recall}{Precision + Recall}$

In [7]:
pred_item = [1, 2, 3, 4, 5]
true_item = [2, 4, 6, 8]

# precision :
# 사용자에게 추천된 상위 K개 아이템 중 실제 사용자가 선호하는 아이템이 얼마나 많은지를 비율로 나타낸 값
# 추천 시스템의 정확도를 평가할 때 유용하게 사용
def precision_at_k(true_item: List[int], pred_item: List[int], k: int) -> float:
    if k == 0:  # 상위 0개의 아이템을 평가 -> 정확도의 의미가 없다.
        return 0.0 # 0.0을 return

     # 실제로 사용자가 선호하고 시스템이 상위 k개로 추천한 아이템의 교집합을 구함
     # 그리고 k로 나누어 상위 k개 추천 중 올바른 추천의 비율을 계산
    p_at_k = (len(set(true_item) & set(pred_item[:k]))) / k
    return p_at_k

print(precision_at_k(true_item, pred_item, 3))

0.3333333333333333


In [8]:
# recall :
# 사용자가 실제로 선호하는 아이템 중에서 시스템이 상위 K개 아이템으로 얼마나 많이 추천했는지를 비율로 나타낸 값
#  추천 시스템이 사용자의 선호를 얼마나 잘 포착하고 있는지를 평가할 때 사용
def recall_at_k(true_item: List[int], pred_item: List[int], k: int) -> float:
    if len(true_item) == 0 or k == 0:
        return 0.0

    r_at_k = (len(set(true_item) & set(pred_item[:k]))) / len(true_item)
    return r_at_k

print(recall_at_k(true_item, pred_item, 3))

0.25


In [9]:
# f1 :
# 정밀도와 재현율의 두 지표를 동시에 고려하여 추천 시스템의 성능을 평가하는 데 사용
# 두 값 모두 높을 때 최대가 되므로, 한 쪽이 아주 높고 다른 쪽이 낮은 경우 보다는 두 지표가 균형있게 높을 때 더 높은 값을 가짐
def f1_at_k(true_item: List[int], pred_item: List[int], k: int) -> float:
    precision = precision_at_k(true_item, pred_item, k)
    recall = recall_at_k(true_item, pred_item, k)

    if precision + recall == 0.0:
        return 0.0

    return 2 * precision * recall / (precision + recall)

print(f1_at_k(true_item, pred_item, 3))

0.28571428571428575


### ROC-AUC, PR-AUC 개념 비교 REMIND
- ROC (Receiver Operating Characteristic Curve) : 민잡도와 1-특이도의 관계를 나타내는 곡선
    - X축 : 거짓 양성 비율 (FPR, False Positive Rate) = $FP / (FP + TN)$
    - Y축 : 참 양성 비율 (TPR, True Positive Rate) = $TP / (TP + FN)$
- ROC-AUC (Area Under the Curve) : ROC 곡선의 아래 면적
    - 0.5 ~ 1사이의 값을 가진다.
    - 모델의 분류 성능을 전체적으로 평가하는 지표, 1에 가까울수록 성능이 좋다.
- PR (Precision-Recall Curve) : Precision과 Recall의 관계를 나타내는 곡선
    - X축 : Recall = $TP / (TP + FN)$
    - Y축 : Precision = $TP / (TP + FP)$
- PR-AUC : PR 곡선의 아래 면적
    - 0과 1사이의 값
    - 모델이 양성 클래스에 대해 얼마나 정확하고 포괄적인지 평가

**한줄 요약**
- ROC-AUC : 모델의 전체적인 분류 능력을 평가, 클래스 불균형에 덜 민감
- PR-AUC : 양성 클래스에 대한 모델의 정밀도와 재현율을 평가하며, 클래스 불균형이 심한 경우에 유용

### 순위 평가 지표
1. MRR (Mean Reciprocal Rank) : 최초의 추천 아이템이 사용자 순위에서 얼마나 상위에 위치하는가를 평가
    - $MRR@K = \frac{1}{|U|} \sum_{u \in U} \frac{1}{k_u}$
    - $U$ : 사용자 전체 집합, $k_u$ : 순위 K번째 안의 최초 적합 위치
2. Average Precision@K (평균 정밀도) : Precision@K의 평균
    - Precision@i는 추천한 아이템 개수 K중에서 해당 인덱스까지만 고려했을 때의 Precision값
    - $AP@K = \frac{1}{\sum_{k=1}^{K} C_{u,k}} \sum_{k=1}^{K} C_{u,k} * Precision@k * rel(k)$
    - $K$ : 순위의 길이 , $C_{u,k}$ : 1 = 사용자 u의 k번째 item이 적합, 0 : 그 외, $rel(k)$ : relevance, 해당 item을 사용자가 좋아했는지 여부 (0 또는 1값),${\sum_{k=1}^{K} C_{u,k}}$ : 모든 아이템 중에서 사용자가 좋아한 아이템 수
    - `precision에 relevence 값을 곱해주는 이유` : 정답 아이템이 추천 목록 중에서 딱 해당 순위에만 영향력을 주도록 하기 위함
3. MAP (Mean Average Precision) : 모든 유저에 대한 Average Precision@K의 평균
    - 사용자가 선호한 아이템이 추천 리스트 중 어떤 순서에 포함되었는지 여부에 대해서 1 또는 0으로만 구분한다.
    - $MAP@K = \frac{1}{|U|} \sum_{u=1}^{|U|} (AP@K)_u$
    - $U$ : 사용자 집합, $|U|$ : 사용자 수 
4.  NDCG@K (Normalized Discounted Cumulative Gain) : Top K개 아이템을 추천하는 경우, 추천 순서에 가중치(relevance)를 두어 평가한다. (NDCG@K값은 1에 가까울수록 좋다)
    - 순서별로 가중치 값(관련도, relevance)을 다르게 적용하여 계산한다.
    - **가장 이상적인 추천 조합 대비 현재 모델의 추천리스트가 얼마나 좋은지를 나타내는 지표**
    
    - (1) Relevance : 사용자가 특정 아이템과 얼마나 관련이 있는지를 나타내는 값  
    - (2) Cumulative Gain(CG) : 추천한 아이템의 Relevance 합  
        - 두 추천 모델이 순서에 관계없이 동일한 아이템 셋을 추천한 경우 두 모델의 CG는 같아진다. 즉, 순서를 고려하지 않은 값
        - $CG@K = \sum_{i=1}^{K} rel(i)$  
    - (3) Discounted Cumulative Gain(DCG) : 추천한 아이템의 Relevance를 순서에 따라 가중치를 준 값
        - 순서에 따른 할인 개념을 도입한 것
        - 추천 아이템의 순서가 뒤에 있을수록 분모가 커짐, 그러나 사용자별로 추천 아이템의 수가 다른 경우 정확한 성능 평가가 어렵다는 한계점
        - $DCG@K = \sum_{i=1}^{K} \frac{rel_i}{\log_2(i+1)}$  
    - (4) Normailzed DCG (NDCG) : DCG의 한계점을 보완, DCG에 정규화를 적용한 것
        - $NDCG@K = \frac{DCG@K}{IDCG@K}$  
        - $IDCG@K$ : 최상의 추천 아이템 순서에 따른 DCG 값

    - Ideal Discounted Cumulative Gain(IDCG) : 최선의 추천을 했을 때 받는 DCG값
        - $IDCG@K = \sum_{i=1}^{K} \frac{rel_{i}^{opt}}{\log_2(i+1)}$
5. Hit Rate@K : 전체 사용자 수 대비 적중한 사용자 수를 의미 (적중률)
    - 1. 사용자가 선호한 아이템 중 1개를 제외
    - 2. 나머지 아이템들로 추천 시스템을 학습
    - 3. 사용자별로 K개의 아이템을 추천하고, 앞서 제외한 아이템 포함되면 Hit이다.
    - 4. 전체 사용자 수 대비 Hit한 사용자 수 비율을 구하면 Hit Rate가 된다.
    - $Hit\ Rate@K = \frac{\# of\ Hit\ Users}{\# of\ Total\ Users}$


6. MAE & RMSE
    - 성능 지표와 달리 `'평점 예측'에 대한 평가 방법`이다.
    - (1) MAE(Mean Absolute Error) : 정답 평점과 예측 평점 간 절대 오차에 대해서 평균을 낸 것
        - MAE는 직관적이고 해석이 용이, 하지만 오차가 큰 이상치에 쉽게 영향을 받는다.
        - $MAE = \frac{1}{|\hat{R}|} \sum_{\hat{r}_{ui} \in \hat{R}} |r_{ui} - \hat{r}_{ui}|$
    - (2) RMSE (Root Mean Square Error) : 오차 제곱의 평균을 내고 루트를 취한 것
        - RMSE는 오차의 제곱을 구하기 때문에 MAE보다 큰 오차에 민감하다.
        - $RMSE = \sqrt{\frac{1}{|\hat{R}|} \sum_{\hat{r}_{ui} \in \hat{R}} (r_{ui} - \hat{r}_{ui})^2}$



In [10]:
# RR@K
def rr_at_k(user_relevance: List[int], k: int) -> float:
    nonzero_indices = np.asarray(user_relevance).nonzero()[0]
    if nonzero_indices.size > 0 and nonzero_indices[0] + 1 <= k:
        return 1.0 / (nonzero_indices[0] + 1.0)
    return 0.0

print(rr_at_k([0, 1, 0], 2))

def mrr_at_k(users_relevance: List[List[int]], k: int) -> float:
    return float(
        np.mean(
            [rr_at_k(user_relevance, k) for user_relevance in users_relevance]
        )
    )

print(mrr_at_k([[1, 0, 0], [0, 1, 0], [0, 0, 1]], 2))

0.5
0.5


In [11]:
# AP@K
def ap_at_k(user_relevance: List[int], k: int) -> float:
    if sum(user_relevance[:k]) == 0:
        return 0.0
    nonzero_indices = np.asarray(user_relevance[:k]).nonzero()[0]
    return sum(
        [sum(user_relevance[: idx + 1]) / (idx + 1) for idx in nonzero_indices]
    ) / sum(user_relevance[:k])

print(ap_at_k([0, 1, 0, 1, 0], 5))

def map_at_k(users_relevance: List[List[int]], k: int) -> float:
    return float(
        np.mean(
            [ap_at_k(user_relevance, k) for user_relevance in users_relevance]
        )
    )

ls = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
print(map_at_k(ls, 3))

0.5
0.611111111111111


In [13]:
# nDCG@K
def dcg_at_k(user_relevance: List[int], k: int) -> float:
    user_relevance = np.asarray(user_relevance, dtype=float)[:k]
    if user_relevance.size == 0:
        return 0.0
    discounts = np.log2(np.arange(2, user_relevance.size + 2))
    return np.sum(user_relevance / discounts)

def ndcg_at_k(user_relevance: List[int], k: int) -> float:
    dcg_max = dcg_at_k(sorted(user_relevance, reverse=True), k)
    if not dcg_max:
        return 0.0
    return dcg_at_k(user_relevance, k) / dcg_max

print(ndcg_at_k([0, 2, 0, 1, 0], 5))

0.6433224083306327


- `AttributeError: np.asfarray was removed in the NumPy 2.0 release. Use np.asarray with a proper dtype instead.` 에러 발생
    - `np.asarray 사용`: `user_relevance = np.asfarray(user_relevance)[:k]`를 `user_relevance = np.asarray(user_relevance, dtype=float)[:k]`로 변경
    - `dtype 지정`: np.asarray 함수에 `dtype=float`를 명시하여 실수형 배열로 변환하도록 했다.

In [14]:
def dcg_at_k(user_relevance: List[int], k: int) -> float:
    # user_relevance를 부동 소수점 배열로 변환하고 처음 k개 요소를 사용
    user_relevance = np.asarray(user_relevance, dtype=np.float64)[:k]
    if user_relevance.size == 0:
        return 0.0
    # DCG 계산: 첫 번째 요소 + 나머지 요소의 가중 합
    return user_relevance[0] + np.sum(
        user_relevance[1:] / np.log2(np.arange(2, user_relevance.size + 1))
    )

def ndcg_at_k(user_relevance: List[int], k: int) -> float:
    # 최대 DCG 계산
    dcg_max = dcg_at_k(sorted(user_relevance, reverse=True), k)
    if not dcg_max:
        return 0.0
    # NDCG 계산: DCG / 최대 DCG
    return dcg_at_k(user_relevance, k) / dcg_max

# 예제 출력
print(ndcg_at_k([0, 2, 0, 1, 0], 5))

0.8333333333333334


### 기타 평가 지표
- 특성 평가
    - 정밀도 기준의 추천 시스템은 filter bubble (정보 여과 현상: 선별된 정보만을 제공) 문제 야기
    - 도메인에 따른 여러가지 특성(인기도, 현재 관심사, 장기적인 선호도)을 고려한 추천
- Coverage
    - $Catalogue = \frac{추천된 item 집합}{전체 item}$
    - $User = \frac{실제 추천이 수행된 사용자 집합}{전체 사용자 집합}$
        - cold start 검출에 사용
        - 초기 사용자로 필터링하여 사용자 coverage를 측정함으로써 초기 사용자에게 어느 정도 추천하였는지를 측정
- Novelty(신규성)
    - 순위에 대한 추천 item이 얼마나 새로운지를 나타내는 지표
    - $Novelty(R) = \frac{\sum_{i \in R} -\log_2 p(i)}{|R|}$
        - $R$ : 순위 , $p(i)$ : 모든 사용자에 대해 과거 item의 추천 확률, 
        - $imp(u, i)$ : 1 = 사용자 u에게 item i가 추천된 경우, 0 = 그 외
- Popularity(인기도)
    - 추천된 item이 얼마나 인기 있는지를 나타내는 지표
    - 추천 시스템 성능 비교 baseline
- Personalization(개인화)
    - 얼마나 개인화된 추천을 제공하는지를 나타내는 지표
- Serendipity(흥미로움)
    - 사용자가 예상하지 못한(의외성) 새로운 정보를 제공하는지를 나타내는 지표
- Diversity(다양성)
    - 추천된 item들이 얼마나 다양한지를 나타내는 지표
    - $Diversity(R) = \frac{\sum_{i \in R} \sum_{j \in R \neq i} sim(i, j)}{|R|(|R| - 1)}$
        - $sim(i, j)$ : item i와 j의 유사도
    - 유사도 거리가 먼 경우 다양성이 커짐

### 평가지표 선정 방법
- informational : 1개 이상의 웹 페이지에서 정보를 얻으려는 의도
    - 이진값 적합성 -> Average Precision
    - 다중값 적합성 -> nDCG
- navidational : 1개의 적합 item을 얻고 싶을 때 특정 사이트를 방문하려는 의도
    - 이진값 적합성 -> Reciprocal Rank
    - 다중값 적합성 -> ERR
- Transactional : 웹을 매개로 한 액션을 실행하려는 의도

### Online 평가
- 새로운 테스트 대상의 추천모델이나 새로운 UI를 일부 사용자에게 실제로 제공하여 실제 사용자에게 미치는 영향을 평가
    - A/B test : 사용자를 기능 변경한 결과를 보여주는 treatment 그룹(실험군)과 변경하지 않은 결과를 보여 주는 control 그룹(대조군)으로 나누어 평가
        - 주의점
            - 그룹 편향 주의 : A/A test를 먼저 실시하여 차이가 없을 경우 실시
            - 로그 혼합 : 학습 데이터 부족으로 모델 A, 모델 B를 사용자 A, 사용자 B가 뒤섞인 경우
            - 집계기간 : 단기간/ 장기간 테스트
    - 평가 지표
        - OEC(Overall Evaluation Criteria) : A/B test의 성공과 실패를 최종적으로 판단하는 지표
            - HEART(Happiness, Engagement, Adotion, Retention, Task Success) 측정
        - 가드레일 지표 : 저하되어서는 안되는 제약 (ex : 페이지 열람수, 서비스 가동률, 응답속도, 수익금 등)
    - interleaving : 평가 시 A/B 테스트처럼 사용자 그룹을 나누지 않고, 평가 대상의 각 순위를 하나의 순위로 섞어 사용자에게 제시
        - 뒤섞인 순위에 대한 클릭으로 원래 순위끼리 평가
            - interleaving : 2개의 순위를 섞어 평가
            - Multileaving : 3개 이상의 순위를 섞어 평가

### 사용자 study
- 사용자 study : 설문조사, 인터뷰, 모델, UI의 정성적인 평가
- 조사설계 : 목적, 대상, 방법, 절차, 결과, 해석, 결론
- ResQue : 사용자 중심 평가 프레임 워크
    - 시스템 품질, 사용자의 신념, 사용자의 태도, 행동 의도 등
